# Тетрадка 5. RL Инструментарий

## Содержание:

1. [Модификация сред: Wrappers](#wrappers)
2. [Логирование Wandb](#logs)
3. [RL Фреймворки](#frameworks)


In [ ]:
# @title Установка зависимостей

try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 43.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 kB 2.8 MB/s eta 0:00:00


In [ ]:
# @title Импортирование зависимостей

import time
import glob
import io
import base64
from IPython import display as ipythondisplay
from IPython.display import HTML
import gymnasium as gym
from gymnasium.wrappers.record_video import RecordVideo
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


def show_video(folder="./video"):
    mp4list = glob.glob(folder + '/*.mp4')
    if len(mp4list) > 0:
        mp4 = sorted(mp4list, key=lambda x: x[-15:], reverse=True)[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

# 1. Модификация сред: Wrappers  <a name = 'wrappers'></a>
Мы знаем, что окружения создаются с помощью команды ``gym.make(<имя среды>)``, но что если мы хотим немного изменить окружение или добавить какой-то дополнительный функционал? Для этого существуют обертки (wrappers). [Уже существующие обертки](https://gymnasium.farama.org/api/wrappers/).


## Reward Wrapper
 Вспомним среду ``Taxi-v3``, предположим, что мы хотим поменять вознаграждения на следующие: 1 за решение задачи, -1 за неправильную посадку/высадку пассажира и 0 во всех остальных случаях. Для этого мы можем воспользоваться ``RewardWrapper``-ом:

In [ ]:
class MyRewardWrapper(gym.RewardWrapper):
    def reward(self, reward):
        if reward == -1:
            return 0
        elif reward == 20:
            return 1
        elif reward == -10:
            return -1
        else:
            raise KeyError

In [ ]:
env = gym.make("Taxi-v3")
env = MyRewardWrapper(env)
observation, info = env.reset()

rewards = set()

while True:
    observation, reward, term, trunc, info = env.step(env.action_space.sample())
    rewards.add(reward)
    if term or trunc:
        break

# выведем все вознаграждения, которые получал агент
print(rewards)
env.close()

{0, -1}


## Time Limit Wrapper

В зависимости от рандома, мы могли получить разные результаты, но обычно это ``{0, -1}``. Откуда такой результат? Ведь среда заканчивается только когда задание выполнено. Все дело во встроенной обертке ограничивающей максимальное количество шагов.

In [ ]:
env = gym.make("Taxi-v3")
print(type(env))
env.close()

<class 'gymnasium.wrappers.time_limit.TimeLimit'>


Можно воспользоваться окружением без этой обертки, вызвав ``.env``:

In [ ]:
env = gym.make("Taxi-v3").env
env = MyRewardWrapper(env)

observation, info = env.reset()
rewards = set()
while True:
    observation, reward, term, trunc, info = env.step(env.action_space.sample())
    rewards.add(reward)
    if term or trunc:
        break

print(rewards)
env.close()

{0, 1, -1}


In [ ]:
term, trunc

(True, False)

А если у нас есть окружение без этого враппера по умолчанию, то можно добавить его вот так:

In [ ]:
env = gym.make("Taxi-v3").env
env = MyRewardWrapper(env)
env = gym.wrappers.TimeLimit(env, max_episode_steps=1)

observation, info = env.reset()
rewards = set()

while True:
    observation, reward, term, trunc, info = env.step(env.action_space.sample())
    rewards.add(reward)
    if term or trunc:
        break

print(rewards)
env.close()

{0}


In [ ]:
term, trunc

(False, True)

Как вы считаете, корректно ли с точки зрения MDP, если среда будет досрочно возвращать ``done=True``?

## Action Wrapper
Представим, что наш водитель находится не в лучшем своем состоянии и независимо от выбора агента, в 50% случаев совершает случайные действий. Сделать среду стохастическои и добиться такого эффекта мы можем, используя ``ActionWrapper``:

In [ ]:
class TaxiRandomActionWrapper(gym.ActionWrapper):
    def __init__(self, env, probability=0.5):
        super().__init__(env)
        self.probability = probability

    def action(self, action):
        if np.random.random() < self.probability:
            return env.action_space.sample()
        else:
            return action

Чтобы проверить, что обертка работает будем выполнять единственное действие:

In [ ]:
env = gym.make("Taxi-v3", render_mode='rgb_array')
env = MyRewardWrapper(env)
env = TaxiRandomActionWrapper(env)
env = RecordVideo(env, f"./video")

observation, info = env.reset()
rewards = set()

while True:
    action = 0
    observation, reward, term, trunc, info = env.step(action)
    rewards.add(reward)
    if term or trunc:
        break

print(rewards)
env.close()
show_video()

Moviepy - Building video /content/video/rl-video-episode-0.mp4.
Moviepy - Writing video /content/video/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/video/rl-video-episode-0.mp4
{0, -1}


## Wrapper

Класс ``gym.Wrapper`` является базовым для всех оберток. Подкласс может переопределить    многие методы для изменения поведения исходной среды, не изменяя при этом ее исходный код. Например, мы можем изменить метод step и добавить, какую-то дополнительную информацию в ``info``:

In [ ]:
class MyWrapper(gym.Wrapper):
    def step(self, action):
        observation, reward, term, trunc, info = self.env.step(action)
        info['Wrapped'] = True
        return observation, reward, term, trunc, info

In [ ]:
env = gym.make("Taxi-v3")
env = MyWrapper(env)

env.reset()
_, _, _, _, info = env.step(env.action_space.sample())

print(info)
env.close()

{'prob': 1.0, 'action_mask': array([0, 1, 0, 1, 0, 0], dtype=int8), 'Wrapped': True}


## Atari Preprocessing


Предварительная обработка для Atari 2600.
Класс ``AtariPreprocessing`` следует рекомендациям статьи: Revisiting the Arcade Learning Environment: Evaluation Protocols and Open Problems for General Agents" Machado et al. (2018).

In [ ]:
env = gym.make("BoxingNoFrameskip-v0")

print(f'Original: {env.observation_space.shape}')

Original: (210, 160, 3)


In [ ]:
env = gym.wrappers.AtariPreprocessing(gym.make("BoxingNoFrameskip-v0"))

print(f'Preprocessed: {env.observation_space.shape}')

Preprocessed: (84, 84)


# 2. Логирование Wandb <a name = 'logs'></a>

Несколько примеров: [Dexterity](https://wandb.ai/site/customers/learning-dexterity-end-to-end-using-weights-biases-reports), [RL Example: Pacman](https://wandb.ai/yashkotadia/rl-example).

Для того, чтобы результаты записывались в облако, нужно залогиниться. Предоставить свой api-key. Самый простой способ - использование консоли:

In [ ]:
if COLAB:
    !pip install wandb --quiet

import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import math
import random

# Start a new run, tracking hyperparameters in config
wandb.init(project="test-drive", config={
    "learning_rate": 0.01,
    "dropout": 0.2,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
})
config = wandb.config

# Simulating a training or evaluation loop
for x in range(50):
    acc = math.log(1 + x + random.random()*config.learning_rate) + random.random() + config.dropout
    loss = 10 - math.log(1 + x + random.random() + config.learning_rate*x) + random.random() + config.dropout
    # Log metrics from your script to W&B
    wandb.log({"acc":acc, "loss":loss}, step=x)
wandb.finish()

wandb: Currently logged in as: arkol. Use `wandb login --relogin` to force relogin


acc,▁▃▄▄▅▄▄▅▅▆▅▅▅▅▅▆▇▆▆▇▆▇▇▇▇▆▇▇▇▇▆▇▇▇▇█████
loss,█▆▆▅▆▄▅▄▄▄▄▃▄▂▃▃▃▃▃▂▃▂▂▃▃▂▂▃▂▂▁▂▂▁▁▂▂▁▂▂
acc,4.81129
loss,6.8239


# 3. RL Фреймворки <a name = 'frameworks'></a>

Переиспользование написанного кода является хорошей практикой написания программ и обучение с подкреплением не является исключением. Существует большое число готовых реализаций RL алгоритмов. Рассмотрим некоторые из них:

## Stable Baselines3


Stable Baselines3 (SB3) is a set of reliable implementations of reinforcement learning algorithms in PyTorch. It is the next major version of [Stable Baselines](https://github.com/hill-a/stable-baselines).

You can read a detailed presentation of Stable Baselines3 in the [v1.0 blog post](https://araffin.github.io/post/sb3/) or our [JMLR paper](https://jmlr.org/papers/volume22/20-1364/20-1364.pdf).


These algorithms will make it easier for the research community and industry to replicate, refine, and identify new ideas, and will create good baselines to build projects on top of. We expect these tools will be used as a base around which new ideas can be added, and as a tool for comparing a new approach against existing ones. We also hope that the simplicity of these tools will allow beginners to experiment with a more advanced toolset, without being buried in implementation details.


**Neural Network Framework:** PyTorch.

**Список алгоритмов:** A2C, DDPG, DQN, HER, PPO, SAC, TD3, etc.

Tutorial: [Stable Baselines3 Tutorial](https://github.com/araffin/rl-tutorial-jnrr19)

In [ ]:
if COLAB:
    !pip install stable-baselines3[extra] --quiet
    !apt install swig
    !pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 3.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (1,721 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 120882 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Sel

In [ ]:
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

# Create environment
env = gym.make("LunarLander-v2", render_mode="rgb_array")

# Instantiate the agent
model = DQN("MlpPolicy", env, verbose=0)
# Train the agent and display a progress bar
model.learn(total_timesteps=int(2e5), progress_bar=True)
# Save the agent
model.save("dqn_lunar")
del model  # delete trained model to demonstrate loading

# Load the trained agent
# NOTE: if you have loading issue, you can pass `print_system_info=True`
# to compare the system on which the model was trained vs the current one
# model = DQN.load("dqn_lunar", env=env, print_system_info=True)
model = DQN.load("dqn_lunar", env=env)

# Evaluate the agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)

Output()

KeyboardInterrupt: ignored

In [ ]:
mean_reward

In [ ]:
env = RecordVideo(gym.make("LunarLander-v2", render_mode='rgb_array'), f"./video")
obs, _ = env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, term, trunc, info = env.step(action)
env.close()

show_video()

## CleanRL

[CleanRL](https://docs.cleanrl.dev/) is a Deep Reinforcement Learning library that provides high-quality single-file implementation with research-friendly features. The implementation is clean and simple, yet we can scale it to run thousands of experiments using AWS Batch.


In [ ]:
! git clone https://github.com/vwxyzjn/cleanrl.git
! pip install -r cleanrl/requirements/requirements.txt
! pip install -r requirements/requirements-atari.txt

In [ ]:
! python cleanrl/cleanrl/dqn_atari.py --env-id BreakoutNoFrameskip-v4 --track --capture_video

/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.ai')`.
Implement

## Rllib

[RLlib](https://docs.ray.io/en/master/rllib.html) $-$ это open source RL фреймворк, который предлагает высокую масштабируемость и унифицированный API. RLlib изначально поддерживает TensorFlow, TensorFlow Eager и PyTorch. Для распараллеливания используется [Ray Project](https://github.com/ray-project/ray).

**Neural Network Framework:** Tensorflow, PyTorch.

**[Список алгоритмов:](https://docs.ray.io/en/master/rllib-algorithms.html)** A2C, A3C, ARS, BC, ES, DDPG, TD3, APEX-DDPG, Dreamer, DQN, Rainbow, APEX-DQN, IMPALA, MAML, MARWIL, MBMPO, PG, PPO, APPO, SAC, LinUCB, LinTS, AlphaZero, QMIX, MADDPG


### PFRL


PFRL $-$ это библиотека глубокого обучения с подкреплением, которая реализует различные современные алгоритмы на Python, с использованием PyTorch. Бывший ChainerRL.

**Neural Network Framework:** PyTorch.

**Список алгоритмов:** DQN, Rainbow, IQN, DDPG, A3C, ACER, PPO, TRPO, TD3, SAC

Быстрый старт: [quickstart.ipynb](https://github.com/pfnet/pfrl/blob/master/examples/quickstart/quickstart.ipynb)

Примеры: [examples](https://github.com/pfnet/pfrl/tree/master/examples)

### Заслуживают упоминания:

* [OpenAI Spinning Up RL](https://github.com/openai/spinningup)
* [LeelaChessZero](https://github.com/LeelaChessZero)
* [Tianshou](https://github.com/thu-ml/tianshou)
* [TF agents](https://github.com/tensorflow/agents)
* [Catalyst-rl](https://github.com/catalyst-team/catalyst-rl)
* [Dopamine](https://github.com/google/dopamine)
* [TRFL](https://github.com/deepmind/trfl)
* [Keras RL](https://github.com/keras-rl/keras-rl)
* [PyTorch-RL](https://github.com/Khrylx/PyTorch-RL)
* [PyMarl](https://github.com/oxwhirl/pymarl)
* [Denny Britz RL](https://github.com/dennybritz/reinforcement-learning)
* [DeepRL-Tutorials](https://github.com/qfettes/DeepRL-Tutorials)
* [RL Adventure](https://github.com/higgsfield/RL-Adventure)
* [FacebookResearch ELF](https://github.com/facebookresearch/ELF)
* [SLM-Lab](https://github.com/kengz/SLM-Lab)
